### Перед началом решения выполните следующую ячейку

In [ ]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

In [12]:
from datetime import datetime
from collections import defaultdict
import re


def parse_log_file(log_file_path):
    grouped = defaultdict(dict)
    with open(log_file_path, 'r') as file:
        for line in file:    
            splited = line.split('|')
            key = datetime.strptime(splited[1].strip(), "%Y-%m-%d %H:%M:%S,%f").date()
            user_id = re.search('( id: )(\d*)', splited[4])
            if user_id is not None and 'Обновляем подписку' in splited[4]:
                grouped[key].setdefault('sub', list()).append(int(user_id.group(2)))
            elif user_id is not None and 'У пользователя' in splited[4]:
                if splited[0].startswith('INFO'):
                    grouped[key].setdefault('exp', list()).append(int(user_id.group(2)))
                else:
                    grouped[key].setdefault('err', list()).append(int(user_id.group(2)))
            elif key not in grouped:
                grouped[key] = {}
    return grouped

### Задача 1

In [ ]:
def count_success_and_failure(log_file_path):
    parsed_log = parse_log_file(log_file_path)

    total_sub = sum(len(v.get('sub', [])) for v in parsed_log.values())
    total_err = sum(len(v.get('err', [])) for v in parsed_log.values())

    return (total_sub - total_err, total_err)

**Проверка**

In [14]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### Задача 2

![image info](chapter14case1task2.png)

**Решением с добавлением записи об обновлении подписки для пользователя**

In [ ]:
def auto_renewal_sub(log_file_path):
    parsed_log = parse_log_file(log_file_path)

    strange_date = datetime.strptime('2022-12-31', "%Y-%m-%d").date()
    parsed_log[strange_date]['sub'].append(666)
    
    values = [len({*v.get('sub', [])}) for v in parsed_log.values()]
    means = []
    medians = []
    for i in range(len(values)):
        current_window = values[:i+1]
        
        # среднее
        mean = sum(current_window) / len(current_window)
        means.append(round(mean, 2))
        
        # медианa
        sorted_window = sorted(current_window)
        if len(current_window) % 2 == 0:
            median = (sorted_window[len(current_window)//2 - 1] + 
                    sorted_window[len(current_window)//2]) / 2
        else:
            median = sorted_window[len(current_window)//2]
        medians.append(int(median))
        
    with open('auto_renewal_sub.txt', 'w') as file:
        file.write(f"Среднее: {means}\n")
        file.write(f"Медиана: {medians}\n")
    

auto_renewal_sub('auto_purchase.log')

**Решение основывающееся на разбоер сообщения формата: "*(\d{4})-(\d{2})-(\d{2}).\*: (\d\*)$*"**

In [66]:
from datetime import datetime
from collections import defaultdict
import re


def auto_renewal_sub(log_file_path):
    grouped = defaultdict(int)
    with open(log_file_path, 'r') as file:
        for line in file:
            splited = line.split('|')
            key = datetime.strptime(splited[1].strip(), "%Y-%m-%d %H:%M:%S,%f").date()
            user_qty = re.search(r'(\d{4})-(\d{2})-(\d{2}).*: (\d*)$', splited[4])
            if user_qty is not None:
                qty_value = re.search(r'(\d*)$', user_qty.group(0))
                grouped[key] = max(grouped[key], int(qty_value.group(0)))            

    values = [*grouped.values()]
    means = []
    medians = []
    for i in range(len(values)):
        current_window = values[:i+1]
        
        # среднее
        mean = sum(current_window) / len(current_window)
        means.append(round(mean, 2))
        
        # медианa
        sorted_window = sorted(current_window)
        if len(current_window) % 2 == 0:
            median = (sorted_window[len(current_window)//2 - 1] + 
                    sorted_window[len(current_window)//2]) / 2
        else:
            median = sorted_window[len(current_window)//2]
        medians.append(int(median))
        
    with open('auto_renewal_sub.txt', 'w') as file:
        file.write(f"Среднее: {means}\n")
        file.write(f"Медиана: {medians}\n")

auto_renewal_sub('auto_purchase.log')

**Проверка**

In [ ]:
!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

In [68]:
import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


### Задача 3

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

In [ ]:
def sub_renewal_by_day(file_path):
    pass

sub_renewal_by_day('auto_purchase.log')

**Проверка**

In [ ]:
!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')